In [2]:
import spacy
import zh_core_web_sm
import en_core_web_sm
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
import re
from spacy.tokens import Doc
from spacy.pipeline import EntityRuler
from spacy import displacy
from spacy.tokenizer import Tokenizer
from spacy.symbols import POS
from spacy.strings import StringStore
from spacy.pipeline import Tagger

In [3]:
#小试牛刀
nlp = zh_core_web_sm.load()
# nlp = Chinese()
print(nlp.pipe_names)
print(nlp.tokenizer)

# # 输出识别出来的实体
# for d in doc.ents:
#     print(d.text, d.label_)

# #？？？待解释
# for tok in doc:
#     print(tok.text, "...", tok.dep_)

# for chunk in doc.noun_chunks:
#     print(chunk.text, chunk.root.text, chunk.root.dep_,
#             chunk.root.head.text)




Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.709 seconds.
Prefix dict has been built successfully.


['tagger', 'parser', 'ner']


In [4]:
# 使用自定义matcher
matcher = Matcher(nlp.vocab)
pattern = [{"TEXT": "住所"}, {"TEXT": "地"}]

# Add the pattern to the matcher
matcher.add("residence", None, pattern)

# Use the matcher on the doc
doc = nlp("住所地在山西省大同市")
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: ['住所地']


In [5]:
# 使用指定的string list作为matcher词库去匹配
party_titles = ['三被告共同委托代理人', '两被申诉人的共同委托代理人', '被上诉人(原审原告、反诉被告)', '上述两被告的共同委托诉讼代理人', '代表人', '公司代表人', '法定代表人', '特别授权被告', '上诉人(原审第三人)', '上述两被告委托代理人', '上述两被告共同委 托诉讼代理人', '再审申请人(原审被告)', '负责人', '委托代理人(特别授权)', '被申请人(一审被告、二审被上诉人)', '委托诉讼代理人(特别授权)', '被告委托代理人', '再审申请人', '诉讼委托代理人', '被告一', '拟稿人', '被上诉人(原审被告)', '上诉人(原审原告)', '被上诉人二(原审被告二)', '两被告共同委托代理人', '上诉人共同委托代理人', '原审被告', '被告二', '上诉人(原审被告、反诉原告)', '三上诉人的共同委托代理人', ' 再审申请人(一审原告)', '申诉人(一审原告、二审上诉人)', '支持起诉人', '两上诉人共同委托的诉讼代理人', '原审第二被告', '被上诉人一(原审被告一)', '一审被告', '原告', '被上诉人共同委托代理人', '原审第三人 ', '两原审被告共同委托代理人', '被告', '上述三上诉人共同委托代理人', '被上诉人(一审原告)', '授权代理人', '诉讼代表人', '被告(反诉原告)', '法人代表', '法人代表人', '法定代表人(负责人)', '两被上诉人共同的委托代理人', '上述两被告的委托 诉讼代理人', '申诉人(一审原告、二审被上诉人)', '(一审被告、二审被上诉人)', '一审第三人', '上诉人', '以上原告共同委托代理人', '以上二被上诉人的共同委托诉讼代理人', '申请再审人(原审被告)', '公益诉讼出庭人', '以上二被告共同委托代理人', '以上三被告共同委托代理人', '上述两被告的共同委托代理人', '以上两被告委托代理人', '被申诉人(一审被告、二审上诉人)', '两被上诉人的共同委托代理人', '二被告之共同委托诉讼代理人', '共同委托诉讼代理人', '两被告的共同委托代理人', '支持起诉机关', '上列两被上诉人委托代理人 ', '上述两被上诉人的共同委托诉讼代理人', '二被告共同委托诉讼代理人', '上列两被上诉人的委托代理人', ' 两被告共同的委托代理人', '上述两被告共同委托代理人', '被上诉人的共同委托诉讼代理人', '以上三被告共同的委托诉讼代理人', '抗诉机关', '被上诉人', '原审第三被告', '上诉人(一审原告)', '二被告共同委托代理人', '以上两被告共同委委托代理人', '两被申诉人共同委托诉讼代理人', '特别授权委托诉讼代理人', '两上诉 人共同委托代理人', '以上两被上诉人共同委托诉讼代理人', '上诉人(原审第一被告)', '申诉人(一审原告、二审上诉人、原再审申请人)', '第二被告', '被申请人(原审被告)', '上述两上诉人共同委托代理人', '被申诉人 (一审被告、二审被上诉人、原再审被申请人)', '再审申请人(一审原告、二审上诉人)', '两上诉人的共同委托代理人', '上列两被告的共同委托代理人', '上列两被告共同委托代理人', '被申请人(一审被告)', '原告共同委托诉讼代理人', '被申请人(一审被告、二审上诉人)', '上诉人共同委托诉讼代理人', '被申诉人(一审被告、二审 被上诉人)', '被上诉人(一审被告)', '上述二上诉人的共同委托诉讼代理人', '被申诉人(原审被告)', '二被上 诉人共同委托诉讼代理人', '上述两上诉人共同的委托诉讼代理人', '第三被告', '再审申请人(一审原告、二审被 上诉人)', '两上诉人共同委托诉讼代理人', '被申请人(一审原告、二审被上诉人)', '申诉人(原审原告)', '两被上诉人共同委托诉讼代理人', '诉讼代理人', '上诉人(一审被告)', '被申请人(原审原告)', '上诉人(原审被告)', '以上二上诉人共同的委托诉讼代理人', '上述两上诉人的共同委托代理人', '两上诉人的共同委托诉讼代理人', '上述两被上诉人委托诉讼代理人', '上上诉人(原审被告)', '第三人', '法定代理人', '被上诉人(原审第三人)', '原告(反诉被告)', '第一被告', '三被上诉人共同委托诉讼代理人', '委托代理人', '被申诉人(一审被告、二审被上诉人、再审被申请人)', '委托代理人(特别授权代理)', '两第三人的共同委托代理人', '被上诉人共同委托代理人(特别授权代理)', '再审申请人(一审被告、二审上诉人)', '以上两被告共同委托 代理人', '以上两被告的委托代理人', '上述两上诉人共同委托诉讼代理人', '上述被上诉人共同委托诉讼代理人 ', '二被委托代理人', '被申请人', '两被告共同委托诉讼代理人', '公益诉讼起诉人', '被上诉人(原审原告)', '上述两被上诉人共同委托代理人', '委托诉讼代理人', '上述被告共同委托诉讼代理人', '以上二被上诉 人委托代理人', '上列两被上诉人的共同委托代理人', '共同委托代理人', '再审申请人(原审原告)', '上述上诉人的委托诉讼代理人', '申诉人(一审原告、二审上诉人、再审申请人)', '二被上诉人的委托代理人', '上述两被上诉人的共同委托代理人', '上列二被告共同委托代理人', '以上两被告共同委托诉讼代理人', '执行事务合伙人', '投资人']

party_infos = ['身份证地址', '联系地址', '通信地址', '现住', '住', '信用代码', '籍贯', '住址', '住址地', '所在地', '户籍所在地', '住所', '营业地', '身份证住址', '港澳证件号码', '原名称', '注册号', '组织机构代码', '非公司私营企业住所', '户籍地址', '身份证号码', '住', '组织机构代码证', '公⺠身份号码', '营业执照', '身份号码', '营业执照注册号', '住所地', '执照注册号', '经营场所', '身份证住址', ' 个体工商户营业执照注册', '机构代码', '户籍住所', '统一信用代码', '原名称', '身份证地址', '经营地址', '身份证登记住址', '组织机构代码证号', '执业证号', '曾用名', '统一社会信用代码', '工商注册号', '身份证号', '注册号', '营业场所', '居⺠身份证', '地址', '现住址', '身份证住址', '经常居住地', '代理权限', '户籍地', '营业执照号码', '公⺠身份证号', '中文名称 ', '住址', '所在地', '户籍地址', '组织机构代码证', '公⺠身份号码', '住所地', '代码', '经营业主', '经营场所', '身份证登记住址', '统一社会信用代码', '地址', '注册号', '营业场所', '居⺠身份证号码', '户籍地', '身份证住址']

party_occupations = ['律师', '经理', '总经理', '董事长', '法务', '无职业', '业主', '员工', '店长', '主管', '经营者', '董事', '职员']

cost_type = ['货款','价款','受理费','赔偿金','公证费','诉讼费','购物款','上诉费','公告费', '运费',\
             '交通费','误工费','打印费','鉴定费','邮寄费','赔偿款','医疗费','购酒款','购药款',\
             '住宿费','产品质量监督检验费','其他费用', '餐饮费', '购货款', '减半', '合计', '共计']

# party_gender = ['男', '女']

In [6]:
# matcher = PhraseMatcher(nlp.vocab)

# Create pattern Doc objects and add them to the matcher
# This is the faster version of: [nlp(country) for country in COUNTRIES]
patterns_ptitles = list(nlp.pipe(party_titles))
patterns_pinfos = list(nlp.pipe(party_infos))
patterns_poccups = list(nlp.pipe(party_occupations))
patterns_pcost_types = list(nlp.pipe(cost_type))
# patterns_genders = list(nlp.pipe(party_gender))
# matcher.add("PARTY_TITLES", None, *patterns_ptitles)
# matcher.add("PARTY_INFOS", None, *patterns_pinfos)
# matcher.add("PARTY_OCCUPS", None, *patterns_poccups)
# matcher.add("PARTY_genders", None, *patterns_genders)

# Call the matcher on the test document and print the result
# matches = matcher(doc)
# print([doc[start:end] for match_id, start, end in matches])

In [8]:
#读取数据
import sys
TEXT = []
with open("/Users/starice/Desktop/partyinfo_type2_2016_1_text.txt", "r") as f:
    for line in f:
        TEXT.append(line)

In [9]:
output_dir="/Users/starice/Desktop/party_title_ner"
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
patterns_ptitles = list(nlp2.pipe(party_titles))
patterns_pinfos = list(nlp2.pipe(party_infos))
patterns_poccups = list(nlp2.pipe(party_occupations))


Loading from /Users/starice/Desktop/party_title_ner


In [10]:
print(nlp.pipe_names)

['tagger', 'parser', 'ner']


In [11]:
#待提取的个人信息有：
# title， 姓名， 出生日期， 性别， 职业， 各类补充信息（住址、代码等）
# 以上除了姓名和title是必须的之外，其他信息都是可有可无，但如果有，一定要详细准确
#如果以逗号分隔的每一小段有多个字典匹配结果，就取最长长度的匹配结果
# TEXT = ["原告: 胡鎏亮,男,1991年12月21日出生,汉族,住重庆市巴南区。", 
# "被告: 重庆好又多百货商业有限公司,住所地重庆市南岸区南坪街道南坪正街1号,组织机构代码73984719-7。",
# "法定代表人: GregoryStephenForan,总经理。",
# "委托代理人: 王志剑,男,1978年3月1日出生,汉族,住山东省诸城市。",
# "原告: 孙文革,男,1971年6月8日,汉族,无职业,住吉林省延吉市公园街园铁委四组。",
# "委托代理人冉某(特别授权): ,重庆睿诚律师事务所律师。", 
# "上诉人(原审被告): 深圳市兴兴酒业贸易有限公司,住所地广东省深圳市罗湖区。", 
# "申诉人（一审原告、二审上诉人）:肖飞,男,1986年2月27日出生,汉族,住湖南省洞口县。", 
# "被申诉人（一审被告、二审上诉人）:深圳市威利文化发展有限公司,住所地在广东省深圳市罗湖区宝岗路裕华工贸大厦1栋四层481。", 
# "法定代表人lll。",
# "法定代表人lll。" ,
# "原告刘想中,男,1976年6月21日出生。", 
# "法定代表人陆兆禧,董事。", 
# "被告浙江天猫网络有限公司,住所地浙江省杭州市余杭区。"]
party_dict = {}
party_dict_list = []
party_title_label = "PARTY_TITLE"
party_title_train_data = [] #("原告: lllll", {"entities": [(0, 3, LABEL)]})

for i in TEXT:
    i = i[:-1] #去掉句号
    ii = i.split(",")
    for iii in ii:
        # print(iii)
        doc = nlp2(iii)

        #出生日期
        re_date = r"\d{2,4}年\d{1,2}月\d{1,2}日"
        date_m = re.findall(re_date, iii)
        if len(date_m) > 0:
            party_dict['birth_date'] = date_m[0]
        #性别
        re_gender = r"(男|女)"
        gender_m = re.findall(re_gender, iii)
        if len(gender_m)>0:
            party_dict['gender'] = gender_m[0]

        #title+姓名
        # print("processing party title\n")
        # party_dict['title'] = "未知"
        # party_dict['name'] = "未知"
        matcher_title = PhraseMatcher(nlp2.vocab)
        # patterns_ptitles = list(nlp2.pipe(party_titles))
        matcher_title.add("PARTY_TITLES", None, *patterns_ptitles)
        matches_title = matcher_title(doc)
        # print(matches_title)
        if len(matches_title) > 0:
            temp_matches = [end-start for match_id, start, end in matches_title]
            # print(matches_title)
            #如果有多个匹配结果，就取长度最长的匹配作为最终结果
            final_index = temp_matches.index(max(temp_matches))
            temp_party_result = [(doc[start:end], (start, end)) for match_id, start, end in matches_title][final_index]
            temp_key = str(temp_party_result[0])
            party_name = str(doc[temp_party_result[1][1]:]).replace(":", "")
            # party_dict[temp_key] = party_name
            party_dict['title'] = temp_key
            party_dict['name'] = party_name
            party_title_train_data.append((iii, {"entities": [(iii.index(temp_party_result[0].text), \
                iii.index(temp_party_result[0].text) + len(temp_party_result[0].text), party_title_label)]}))

        #职业
        # party_dict['occupation'] = "未知"
        # print("processing occupation\n")
        matcher_occup = PhraseMatcher(nlp.vocab)
        # patterns_poccups = list(nlp.pipe(party_occupations))
        matcher_occup.add("PARTY_OCCUPS", None, *patterns_poccups)
        matches_occup = matcher_occup(doc)
        if len(matches_occup) > 0:
            temp_matches = [end-start for match_id, start, end in matches_occup]
            final_index = temp_matches.index(max(temp_matches))
            temp_party_result =  [(doc[start:end], (start, end)) for match_id, start, end in matches_occup][final_index]
            party_dict['occupation'] = str(temp_party_result[0])
        
        #其他补充信息
        # print("processing party info\n")
        matcher_info = PhraseMatcher(nlp.vocab)
        # patterns_pinfos = list(nlp.pipe(party_infos))
        matcher_info.add("PARTY_INFOS", None, *patterns_pinfos)
        matches_info = matcher_info(doc)
        if len(matches_info) > 0:
            temp_matches = [end-start for match_id, start, end in matches_info]
            final_index = temp_matches.index(max(temp_matches))
            temp_party_result =  [(doc[start:end], (start, end)) for match_id, start, end in matches_info][final_index]
            temp_key = str(temp_party_result[0])
            temp_info = str(doc[temp_party_result[1][1]:])
            party_dict[temp_key] = temp_info
    
    # print(party_dict)
    # print()
    # print(party_title_train_data)
    # print()
    party_dict_list.append(party_dict)
    party_dict = {}



In [12]:
# output the training data for ner model
with open("/Users/starice/Desktop/partytitle_train_第二类_2016_11_text.txt", "w", encoding="UTF-8") as f:
    for i in party_title_train_data:
        f.write(str(i) + "\n")

In [ ]:
# test the party title ner model we trained in train_ner.py and saved to party_title_ner dir
test_text = [
"原告: 胡鎏亮,男,1991年12月21日出生,汉族,住重庆市巴南区。", 
"被告: 重庆好又多百货商业有限公司,住所地重庆市南岸区南坪街道南坪正街1号,组织机构代码73984719-7。",
"法定代表人: GregoryStephenForan,总经理。",
"委托代理人: 王志剑,男,1978年3月1日出生,汉族,住山东省诸城市。",
"原告: 孙文革,男,1971年6月8日,汉族,无职业,住吉林省延吉市公园街园铁委四组。",
"委托代理人冉某(特别授权): ,重庆睿诚律师事务所律师。", 
"上诉人(原审被告): 深圳市兴兴酒业贸易有限公司,住所地广东省深圳市罗湖区。", 
"申诉人（一审原告、二审上诉人）:肖飞,男,1986年2月27日出生,汉族,住湖南省洞口县。", 
"被申诉人（一审被告、二审上诉人）:深圳市威利文化发展有限公司,住所地在广东省深圳市罗湖区宝岗路裕华工贸大厦1栋四层481。", 
"法定代表人lll。",
"法定代表人lll。" ,
"原告刘想中,男,1976年6月21日出生。", 
"法定代表人陆兆禧,董事。", 
"被告浙江天猫网络有限公司,住所地浙江省杭州市余杭区。"
]

output_dir="/Users/starice/Desktop/party_title_ner"
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
patterns_ptitles = list(nlp2.pipe(party_titles))
patterns_pinfos = list(nlp2.pipe(party_infos))
patterns_poccups = list(nlp2.pipe(party_occupations))




In [ ]:
for i in test_text:
    ii = i.split(",")
    for iii in ii:
        doc2 = nlp2(iii)
        print(doc2.text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)
        matcher_title = PhraseMatcher(nlp2.vocab)
        matcher_title.add("PARTY_TITLES", None, *patterns_ptitles)
        matches_title = matcher_title(doc2)
        if len(matches_title) > 0:
            temp_matches = [end-start for match_id, start, end in matches_title]
            # print(matches_title)
            #如果有多个匹配结果，就取长度最长的匹配作为最终结果
            final_index = temp_matches.index(max(temp_matches))
            temp_party_result = [(doc2[start:end], (start, end)) for match_id, start, end in matches_title][final_index]
            temp_key = str(temp_party_result[0])
            party_name = str(doc2[temp_party_result[1][1]:]).replace(":", "")
#             print(temp_key, party_name)
            # party_dict['title'] = temp_key
            # party_dict['name'] = party_name
            # party_title_train_data.append((iii, {"entities": [(iii.index(temp_party_result[0].text), \
            #     iii.index(temp_party_result[0].text) + len(temp_party_result[0].text), party_title_label)]}))
#         print()

<h3>Dependency parsing on judgement result text</h3>

In [ ]:
# with open("/Users/starice/Desktop/token_dict_第二类_2016_8.txt", "r", encoding="UTF-8") as f:
#     tokens_dict = eval(f.read())
# print(tokens_dict)

In [ ]:
#将原告被告信息添加至词库做实体识别
PARTY_TITLE_TEXT = []
party = []
with open("/Users/starice/Desktop/partyinfo_result_第二类_2016_8_text.txt", "r", encoding="UTF-8") as f:
    for line in f:
        PARTY_TITLE_TEXT.append(line)

In [ ]:
for i in PARTY_TITLE_TEXT:
    if "{" in i:
        temp_party_dict = eval(i)
        if "name" in temp_party_dict:
            party.append(temp_party_dict['name'])

In [ ]:
# Append words to user dict(like jieba.add_word) 帮助分词， 抛弃原先用jieba分词结果代替spacy分词的方法
nlp.tokenizer.pkuseg_update_user_dict(party)
nlp.tokenizer.pkuseg_update_user_dict(cost_type)
nlp.tokenizer.pkuseg_update_user_dict(party_titles)
nlp.tokenizer.pkuseg_update_user_dict(party_infos)
#细微分词调节
judgeresult_verbs = ["赔偿", "退还", "负担", "返还", "支付", "赔付", "承担", "收取", "迳付", "给付", "交纳", "缴纳"]
nlp.tokenizer.pkuseg_update_user_dict(judgeresult_verbs)
nlp.tokenizer.pkuseg_update_user_dict(["本判决", "发生法律效力之日起", "发生法律效力后", \
                                       "本判决生效之日起", "本判决生效后", "于", "内", \
                                       "向", "计", "张**"])

In [ ]:
print(nlp.pipe_names)

In [ ]:
# 使用entityruler帮助nlp识别特定的实体(费用类型、当事人等等)

# 原先使用的是entity matcher 但是会有overlapping错误
# class EntityMatcher(object):
#     name = "entity_matcher"

#     def __init__(self, nlp, terms, label, matcher_name):
#         patterns = [nlp.make_doc(text) for text in terms]
#         self.matcher = PhraseMatcher(nlp.vocab)
#         self.matcher.add(label, None, *patterns)
#         self.name = matcher_name

#     def __call__(self, doc):
#         matches = self.matcher(doc)
#         for match_id, start, end in matches:
#             span = Span(doc, start, end, label=match_id)
#             doc.ents = list(doc.ents) + [span]
#         return doc
# entity_matcher = EntityMatcher(nlp, cost_type, "COST_TYPE", "entity_matcher_costtype")
# nlp.add_pipe(entity_matcher, after="ner")
# entity_matcher_party = EntityMatcher(nlp, party, "PARTY", "entity_matcher_party")
# nlp.add_pipe(entity_matcher_party, after="ner")

entityruler = EntityRuler(nlp, overwrite_ents=True)
patterns_cost_type = [{"label": "COST_TYPE", "pattern": str(i)} for i in cost_type]
patterns_party = [{"label": "PARTY", "pattern": str(i)} for i in party]
patterns_party_title = [{"label": "PARTY_TITLE", "pattern": str(i)} for i in party_titles]
patterns_party_info = [{"label": "PARTY_INFO", "pattern": str(i)} for i in party_infos]
patterns_judgere_verbs = [{"label": "JUDGE_VERB", "pattern": str(i)} for i in judgeresult_verbs]
entityruler.add_patterns(patterns_cost_type + patterns_party \
                         + patterns_party_title + patterns_party_info \
                        + patterns_judgere_verbs)
nlp.add_pipe(entityruler, last=True)

In [ ]:
#使用regex来帮助识别金额实体
money_re = r"(?P<money_amount>([零一二三四五六七八九十百千万亿角元\d+\.,]*)(?=[角|元])[角|元])"

animal_hash = StringStore([u'MONEY_AMOUNT']) # <-- match id
nlp.vocab.strings.add('MONEY_AMOUNT')

class MoneyRecognizer(object):
    """Example of a spaCy v2.0 pipeline component that sets entity annotations
    based on list of single or multiple-word company names. Companies are
    labelled as ORG and their spans are merged into one token. Additionally,
    ._.has_tech_org and ._.is_tech_org is set on the Doc/Span and Token
    respectively."""

    name = "money_amount"  # component name, will show up in the pipeline

    def __init__(self, nlp, label="MONEY_AMOUNT"):
        """Initialise the pipeline component. The shared nlp instance is used
        to initialise the matcher with the shared vocab, get the label ID and
        generate Doc objects as phrase match patterns.
        """
        self.label = nlp.vocab.strings[label]  # get entity label ID

    def __call__(self, doc):
        """Apply the pipeline component on a Doc object and modify it if matches
        are found. Return the Doc, so it can be processed by the next component
        in the pipeline, if available.
        """
        
        expression = money_re
        spans = []  # keep the spans for later so we can merge them afterwards
        for match in re.finditer(expression, doc.text):
#             print(match)
            start, end = match.span()
#             print(start, end)
#             print(doc)
            matched_span = doc.char_span(start, end, label=self.label)
#             print("matched_span", matched_span)
            # This is a Span object or None if match doesn't map to valid token sequence
            if matched_span is not None:
                spans.append(matched_span)
                doc.ents = list(doc.ents) + [matched_span]
                for s in spans:
                    s.merge
        return doc
                

In [ ]:
money_component = MoneyRecognizer(nlp)  # initialise component
nlp.add_pipe(money_component, first=True)

In [ ]:
#使用自定义的pos mapping table来对一些词汇指定词性
tagger = Tagger(nlp.vocab)
tagger.add_label("退还", {POS: 'VERB'})
tagger.add_label("赔偿", {POS: 'VERB'})
tagger.add_label("负担", {POS: 'VERB'})
tagger.add_label("返还", {POS: 'VERB'})
tagger.add_label("支付", {POS: 'VERB'})
tagger.add_label("赔付", {POS: 'VERB'})
tagger.add_label("承担", {POS: 'VERB'})
tagger.add_label("收取", {POS: 'VERB'})
tagger.add_label("迳付", {POS: 'VERB'})
tagger.add_label("给付", {POS: 'VERB'})
tagger.add_label("向", {POS: 'VERB'})
tagger.add_label("受理费", {POS: 'NOUN'})
tagger.add_label("深圳乐荣超市有限公司", {POS: 'NOUN'})

In [ ]:
#将 entity识别结果应用到分词
merge_nps = nlp.create_pipe("merge_noun_chunks")
nlp.add_pipe(merge_nps)
merge_ents = nlp.create_pipe("merge_entities")
nlp.add_pipe(merge_ents) #merge一般都在ner之后~

In [ ]:
# nlp.disable_pipes("ner")
print(nlp.pipe_names)

In [ ]:
# temp_t = '案件受理费50元,减半收取计25元,由被告天津市人人乐商业有限公司、被告天津市人人乐商业有限公司河西购物广场承担。'
# temp_tt = temp_t.split(",")
# for tt in temp_tt:
#     doc_dep = nlp(tt)
#     displacy.serve(doc_dep, style="dep")
doc_dep = nlp("悦家公司阳光店于本判决发生法律效力之日起10日内向朱玉娄支付赔偿金2500元;悦家公司阳光店就上述付款不能履行部分由悦家公司承担连带清偿责任。")


displacy.render(doc_dep, style="ent")

In [ ]:
# for tt in temp_tt:
#     doc_dep = nlp(tt)
#     for token in doc_dep:
#         print(token.text, token.pos_, token.dep_, token.head.text, token.head.pos_,
#                 [child for child in token.children])
for token in doc_dep:
        print(token.text, token.pos_, token.dep_, token.ent_type_, token.head.text, token.head.pos_,
                [child for child in token.children])
# print(set(doc_dep))
        
for chunk in doc_dep.noun_chunks:
    print(chunk.text)
#           , chunk.root.text, chunk.root.dep_,
#             chunk.root.head.text)

In [1]:
#系统的提取金额类型，数目，承担人或受益人
JUDGERE_TEXT = []
with open("/Users/starice/Desktop/judgeresult_第二类_2016_8_text.txt", "r", encoding="UTF-8") as f:
    for line in f:
        JUDGERE_TEXT.append(line)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/starice/Desktop/judgeresult_第二类_2016_8_text.txt'

In [ ]:
def isInter(a,b):
    result = list(set(a)&set(b))
    if result:
        return True
    else:
        return False

def extract_loader(doc):
    verb_1 = ["承担", "负担", "交纳", "缴纳"]
    verb_2 = ["赔偿", "退还", "返还", "支付", "赔付", "收取", "迳付", "给付"]
    loader = []
    if isInter(verb_1, [str(i) for i in doc]):
        print("processing verb_1 type...")
        for token in doc:
#             print(token.text, token.dep_, token.ent_type_, [child for child in token.children])
            if token.dep_ == "nsubj" and token.ent_type_ == "PARTY":
                return [token.text]
            if (token.dep_ == "nmod:prep" and token.ent_type_ == "PARTY") or \
            (token.dep_ == "conj" and token.ent_type_ == "PARTY") or \
            (token.dep_ == "compound:nn" and token.ent_type_ == "PARTY"):
                loader.append(token.text)
            
        if len(loader) == 0: #关键字匹配
#             print("processing verb_1 type keyword match...")
            party_changed = False
            for token in doc:
                if token.ent_type_ == "PARTY_TITLE":
                    temp_loader = token.text
                    party_changed = True
                if token.ent_type_ == "JUDGE_VERB" and token.text in verb_1:
#                     print("temp_loader:", temp_loader)
#                     print("party_changed:", party_changed)
                    if party_changed:
                        loader.append(temp_loader)
                        party_changed = False
            loader = list(set(loader))
    else:
        print("processing verb_2 type...")
        temp_nsubj = []
        #如果句子开头是个介词（例如限），则找ent type 是 party 或 party_title 的 dobj
        if doc[0].dep_ == "dep":
            for token in doc:
                if token.dep_ == "dobj":
                    if token.ent_type_ == "PARTY" or token.ent_type_ == "PARTY_TITLE" \
                    or token.ent_type_ in ["PARTY", "PARTY_TITLE", "ORG", "PROPN", "PERSON"]:
                        temp_nsubj.append(token.text)
        else:
            for token in doc:
                if token.dep_ == "nsubj":
                    if token.ent_type_ == "PARTY" or token.ent_type_ == "PARTY_TITLE" \
                    or token.ent_type_ in ["PARTY", "PARTY_TITLE", "ORG", "PROPN", "PERSON"]:
                        temp_nsubj.append(token.text)
        print(temp_nsubj)
        if len(temp_nsubj) == 0: 
#             print("processing verb_2 type keyword match...")
            for token in doc:
                pass
        elif len(temp_nsubj) == 1:
            loader.append(temp_nsubj[0])
            return loader
        else:
            print("multiple nsubj, split the sentence by ','")
            sents = doc.text.split(",")
            for i in sents:
                if re.search(money_re, i) is not None:
                    print(i)
                    temp_doc = nlp(i)
                    for temp_token in temp_doc:
                        print(temp_token.text, temp_token.dep_, temp_token.ent_type_)
                        if temp_token.dep_ == "nsubj" and temp_token.ent_type_ in ["PARTY", \
                                                                                   "PARTY_TITLE", "ORG", "PROPN", "PERSON"]:
                            loader.append(temp_token.text)
                            break
            if len(loader) == 0:
                loader = temp_nsubj[0]
                    
                
    return loader
            
            

In [ ]:
# 处理 judgement result的方法
def extract_juegement_result(t):
    t = t.replace("\n", "")
    t = t[:-1] #去掉最后一个符号
    t = re.sub(r'[一二三四五六七八九十]、', '', t)
    if re.search(money_re, t) is None: return
    else:
#         print(t)
        if "。" in t: sents = t.split("。")
        elif ";" in t: sents = t.split(";")
        else: sents = [t]
        final_result = []
        for i in sents:
            print(i)
            #这里先好好处理单个句子，不考虑是针对某个案件的，先把不同种类的句子都处理好
            #先提取(cost_type, money_amount)
            temp_extracted_infos = [] #example: [('受理费', '25元', '重庆永辉超市有限公司')]
            judgere_doc = nlp(i)
            if re.search(money_re, judgere_doc.text) is not None:
                # 输出识别出来的实体
#                 print(judgere_doc.text)
                has_cost_type = False
                for d in judgere_doc.ents:
                    print(d.text, d.label_)
                    if d.label_ == "COST_TYPE":
                        has_cost_type = True
                        #temp_cost_type.append(d.text)
                        temp_extracted_info = [d.text, '', '']
                    if d.label_ == "MONEY_AMOUNT":
                        if has_cost_type:
                            temp_extracted_info[1] = d.text
                            has_cost_type = False
                        else:
                            temp_extracted_info = ["统一赔偿费用", d.text, '']
                        temp_extracted_infos.append(temp_extracted_info)
                print("temp_extracted_infos", temp_extracted_infos)
                
                loader = extract_loader(judgere_doc)
                for tr in temp_extracted_infos:
                    tr[2] = loader
                final_result += temp_extracted_infos
    return final_result
            

In [ ]:
for i in JUDGERE_TEXT[0:1]:
    i = "二、被告深圳市正德一品贸易商行应于本判决生效之日起五日内支付原告王福群购物价款十倍的赔偿金人民币35800元。"
    print(i)
    print(extract_juegement_result(i))

In [ ]:
#一个简单的信息提取示例小程序
nlp = en_core_web_sm.load()
text = "The Patient report is Positive for ABC disease"
doc = nlp(text)
tokens = {token.text:token for token in doc}
print(tokens)
print()

def is_attribute(token):
    # todo: use a classifier to determine whether the token is an attrubute
    return token.pos_ == 'ADJ'

def bfs(token, predicate, max_distance=3):
    queue = [(token, 0)]
    while queue:
        print(queue)
        t, dist = queue.pop(0)
        print(t, dist)
        if max_distance and dist > max_distance:
            return
        if predicate(t):
            return t
        # todo: maybe, consider only specific types of dependencies or tokens
        neighbors =  [t.head] + list(t.children)
        print("neighbors: ", neighbors, "\n")
        for n in neighbors:
            if n and n.text:
                queue.append((n, dist+1))

print(bfs(tokens['ABC'], is_attribute))  # Positive